In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from zipfile import ZipFile
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_data = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv')
sample_data.head()

In [ ]:
test = sample_data['text']
test.head()

In [ ]:
lis = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
lis2 = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv')
lis.head()

In [ ]:
lis['compound'] = lis.sum(axis = 1)
def a(x) :
    if x> 0:
        return 1
    else:
        return 0
lis['y'] = lis['compound'].apply(lambda x: a(x))
lis.head()

In [ ]:

lis = lis[['comment_text', 'y']]
lis.head()

In [ ]:
less_arr = pd.DataFrame(lis2['less_toxic']).rename(columns = {'less_toxic':'comment_text'}, inplace = False)
more_arr = pd.DataFrame(lis2['more_toxic']).rename(columns = {'more_toxic':'comment_text'}, inplace = False)
more_arr['y'] = 1
less_arr['y'] =  0  
lis  = pd.concat([less_arr, more_arr, lis])
lis.tail()

In [ ]:
lis.shape

In [ ]:
contractions_dict = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews
lis['comment_text']= lis['comment_text'].apply(lambda x:expand_contractions(x))

lis.head()

In [ ]:
test= test.apply(lambda x:expand_contractions(x))

In [ ]:
lis['comment_text'] = lis['comment_text'].str.lower()

In [ ]:
test = test.str.lower()

In [ ]:
import string
lis['comment_text'] = lis['comment_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
lis.sample(5)

In [ ]:
test = test.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

In [ ]:
lis['comment_text'] = lis['comment_text'].apply(lambda x: re.sub('\W+',' ', x))
lis.head()

In [ ]:
test = test.apply(lambda x: re.sub('\W+',' ', x))

In [ ]:
lis[lis['y']>=0].hist()

In [ ]:
lis = pd.concat([lis[lis['y'] > 0], lis[lis['y'] == 0].sample(int(len(lis[lis['y'] > 0])*1.5))], axis = 0).sample(frac = 1)

In [ ]:
lis[lis['y']>=0].hist()

In [ ]:
lis.shape

In [ ]:
import unidecode
lis['comment_text'] = lis['comment_text'].apply(lambda x: unidecode.unidecode(x))

In [ ]:
test = test.apply(lambda x: unidecode.unidecode(x))

In [ ]:
import textblob as tb
from textblob import TextBlob
def sentiment(text):
    tb = TextBlob(text)
    tb = (tb.sentiment.polarity-1)/(-2)
    return tb
lis['senti'] = lis['comment_text'].apply(lambda x: sentiment(x))
sns.heatmap(lis.corr(), annot = True)

In [ ]:

sample_data['senti'] = sample_data['text'].apply(lambda x: sentiment(x))

In [ ]:
lis = lis.sample(frac = 1)
text = list(lis['comment_text'])
submit = list(sample_data['text'])
outputs = list(lis['y'])
sentimentX = list(lis['senti'])
sentimentY = list(sample_data['senti'])
for i in test:
    submit.append(i)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = 250000)
t_samples = 100000
v_samples = 15832
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
submit_sequences = tokenizer.texts_to_sequences(submit)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))
data = pad_sequences(sequences, maxlen = 1000)
data_submit = pad_sequences(submit_sequences, maxlen = 1000)
outputs = np.asarray(outputs)
print('shape of data tensor', (data.shape, data_submit.shape))
print('shape of output', outputs.shape)

In [ ]:
indices = np.arange(data.shape[0])
data = data[indices]
labels = outputs[indices]
Xt, Xv = data[:t_samples], data[t_samples: t_samples+v_samples]
yt, yv = outputs[:t_samples], outputs[t_samples: t_samples+v_samples]
St, Sv = sentimentX[:t_samples], sentimentX[t_samples: t_samples+v_samples]

In [ ]:
f = open(os.path.join('/kaggle/input/glove-global-vectors-for-word-representation', 'glove.twitter.27B.100d.txt'))
embeddings_index = {}
for line in f:
    value = line.split()
    word = value[0]
    coefs = np.asarray(value[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors'%len(embeddings_index))

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((1193514, embedding_dim))
for word, i in word_index.items():
    if i<200:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
embedding_matrix

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Embedding, Flatten, Dense, Dropout, GRU, concatenate, Conv1D, MaxPooling1D
text_input = Input(shape = (1000, ))
text_input_layer = Embedding(1193514, 100, input_length = 1000, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable = True)(text_input)
model = Conv1D(32, 7, activation = 'relu')(text_input_layer)
model = LSTM(64)(model)
lrelu = lambda x: tf.keras.layers.LeakyReLU( alpha = 0.01)(x)
senti_input = Input(shape = (1, ))
senti_input_layer = Dense(32, activation = lrelu)(senti_input)
model = concatenate([model, senti_input_layer])
model = Dense(128, activation= lrelu)(model)
model = Dropout(0.08)(model)
output = Dense(1, activation = 'sigmoid')(model)

model = keras.Model(inputs=[text_input, senti_input], outputs= output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit([Xt, np.asarray(St)], yt, epochs = 4, batch_size = 32, validation_data = ([Xv, np.asarray(Sv)], yv))


In [ ]:
predictions = model.predict(data_submit)

In [ ]:
sample_data['score'] = predictions
sample_data.head()
del sample_data['text']

In [ ]:
sample_data.to_csv('submission.csv', index = False, float_format='%.20f' )
